# PIK3CA: Effects of Mutation

<b> Standard imports for playing with and plotting data frames. </b>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import re
#import altair as alt

#alt.renderers.enable('notebook')


<b> Import CPTAC </b>

In [2]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


In [3]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

<b> List of proteins that interact with PIK3R1 (according to STRING) </b>

In [4]:
protList = ['IRS1', 'IRS2', 'AKT1', 'AKT2', 'CDK', 'P55G', 'ERBB3', 'P85A', 'MRAS', 'NRAS', 'RRAS', 'PDK1', 'CDK1', 'PTEN','PIK3R1']

<b> Proteome abundance of proteins that interact with PIK3R1 </b>

In [5]:
pcutoff = 0.05 / len(protList)
gene = 'PIK3CA'

sigResults = [];
for protein in protList: 
    if protein in proteomics.columns:
        cross = CPTAC.merge_mutations(proteomics, protein, gene)
        cross = cross[["Mutation", protein]].dropna(axis=0)
        mutated = cross.loc[cross["Mutation"] != "Wildtype"]
        wt = cross.loc[cross["Mutation"] == "Wildtype"]
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        if(ttest[1] <= pcutoff):
            sigResults.append(protein)
            print("Test for " + protein + ": ")
            print(ttest) 

AttributeError: module 'CPTAC' has no attribute 'merge_mutations'

<b> Print the list of proteins with significant results </b>

In [ ]:
print(sigResults)

### Phosphoproteome abundance of interacting proteins

In [ ]:
phosProtList = ['IRS1', 'IRS2', 'AKT1', 'AKT2', 'P55G', 'ERBB3', 'P85A', 'MRAS', 'PTEN', 'RPS6KB1', 'PI3KR1']

In [ ]:
sites = phos.columns;
significantPhosResults = [];
totalSites = 0

for protein in phosProtList:
    matchesdf = phos.filter(regex=protein)
    totalSites += len(matchesdf.columns)

pcutoff = 0.05 / totalSites
print(pcutoff)
#pcutoff = 1
    
    

for protein in phosProtList:
    pattern = re.compile(protein)
    isInList = filter(pattern.search, sites)
    if next(isInList, None) is not None:
        phosphositesdf = CPTAC.merge_mutations(phos, protein, gene)
        for site in phosphositesdf.columns:
            if (site is not 'Mutation'):
                sitedf = CPTAC.merge_mutations(phos, site, gene)
                mutateddf = sitedf.loc[sitedf['Mutation'] == 'Missense_Mutation'].dropna(axis=0)
                wtdf = sitedf.loc[sitedf['Mutation'] == 'Wildtype'].dropna(axis=0)
                ttest = scipy.stats.ttest_ind(mutateddf[site], wtdf[site])
                if(ttest[1] <= pcutoff):
                    significantPhosResults.append(site)
                    print("Test for " + site + ": ")
                    print(ttest)   

## Let's investigate another protein that is involed with the PI3K signal pathway: PDK1

 One study shows that IRS1 recruits PDK1, which plays an important role in regulating regeneration and degeneration of PIP3 (https://academic.oup.com/mend/article/21/9/2294/2738656)